# [TR-004] Analyticity (WIP)

<!-- cspell:ignore matexpr -->

```{warning}
This report is Work-in-Progress.
```

Investigation of Section 2.1.2 in {cite}`aitchisonUnitarityAnalyticityCrossing2015`.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib widget
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.dynamics import phase_space_factor
from matplotlib import cm
from mpl_interactions import heatmap_slicer

warnings.filterwarnings("ignore")

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

In [ ]:
s = sp.Symbol("s")
m1, m2 = sp.symbols("m1 m2", real=True)
rho = 16 * sp.pi * phase_space_factor(s, m1, m2)
rho

Or, assuming both decay products to be of unit mass:

In [ ]:
rho.subs(
    {
        m1: 1,
        m2: 1,
    }
)

In [ ]:
np_rho = sp.lambdify((s, m1, m2), rho, "numpy")

m1_val = 1.8
m2_val = 0.5
s_thr = (m1_val + m2_val) ** 2
s_diff = abs(m1_val - m2_val) ** 2

x = np.linspace(-1, +7, num=100)
y = np.linspace(-2, +2, num=100)
X, Y = np.meshgrid(x, y)
s_values = X + Y * 1j
rho_values = np_rho(s_values, m1=m1_val, m2=m2_val)

In [ ]:
rho_min, rho_max = -5, +5
Z = rho_values.real
fig, ax = plt.subplots(
    figsize=(10, 10),
    subplot_kw={"projection": "3d"},
    tight_layout=True,
)
fig.suptitle(R"Solution sheet for $\rho$ in the $s$-plane")
colors = np.select(
    [Z < rho_min, Z < 0, Z >= 0, Z > rho_max],
    [rho_min, Z / np.abs(rho_min), Z / np.abs(rho_max), rho_max],
)
colors = (colors + 1) / 2
ax.plot_surface(
    X,
    Y,
    Z,
    facecolors=cm.bwr(colors),
    linewidth=0,
)
ax.set_xlabel("Re($s$)")
ax.set_ylabel("Im($s$)")
ax.set_zlabel(R"Re($\rho$)")
ax.set_zlim(rho_min, rho_max)
ax.view_init(elev=25, azim=-110)
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "004-3D-plot.svg"
    plt.savefig(output_file)
    display(SVG(output_file))

In [ ]:
fig, axes = heatmap_slicer(
    x,
    y,
    (rho_values.real, rho_values.imag),
    heatmap_names=(R"Re($\rho$)", R"Im($\rho$)"),
    labels=("Re($s$)", "Im($s$)"),
    interaction_type="move",
    slices="both",
    vmin=-5,
    vmax=5,
    figsize=(12, 3),
)
for ax in axes[2:]:
    ax.set_ylim(rho_min, rho_max)
    tick_width = 5
    tick_min = np.around(rho_min / tick_width, decimals=0) * tick_width
    ax.set_yticks(np.arange(tick_min, rho_max + 0.1, 5))
axes[2].set_title("Re($s$)")
axes[3].set_title("Im($s$)")
for ax in axes[:3]:
    ax.axvline(s_diff, c="black", linewidth=0.3, linestyle="dotted")
    ax.axvline(s_thr, c="black", linewidth=0.3, linestyle="dotted")
for ax in axes:
    ax.axvline(0, c="black", linewidth=0.5)
    ax.axhline(0, c="black", linewidth=0.5)
axes[3].axvline(0, c="black", linewidth=0.5)
plt.show()

{{ run_interactive }}

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "004-interactive-locator.svg"
    plt.savefig(output_file)
    display(SVG(output_file))